In [1]:
## Imports

# utility modules
import glob
import os
import sys
import re

# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt

# specialty modules
import h5py
import pyproj

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [5]:
! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
sys.path.append(os.path.join(os.getcwd(), '../..'))
import pointCollection as pc

In [6]:
from readers.read_HDF5_ATL03 import read_HDF5_ATL03
from readers.get_ATL03_x_atc import get_ATL03_x_atc

In [7]:
data='/srv/tutorial-data/land_ice_applications/'

In [8]:
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

In [11]:
def plot_segs(D6, ind=None, **kwargs):
    """
    Plot a sloping line for each ATL06 segment
    """
    if ind is None:
        ind=np.ones_like(D6['h_li'], dtype=bool)
    #define the heights of the segment endpoints.  Leave a row of NaNs so that the endpoints don't get joined
    h_ep=np.zeros([3, D6['h_li'][ind].size])+np.NaN
    h_ep[0, :]=D6['h_li'][ind]-D6['dh_fit_dx'][ind]*20
    h_ep[1, :]=D6['h_li'][ind]+D6['dh_fit_dx'][ind]*20
    # define the x coordinates of the segment endpoints
    x_ep=np.zeros([3,D6['h_li'][ind].size])+np.NaN
    x_ep[0, :]=D6['x_atc'][ind]-20
    x_ep[1, :]=D6['x_atc'][ind]+20

    plt.plot(x_ep.T.ravel(), h_ep.T.ravel(), **kwargs)

In [12]:
def min_seg_difference(D6):
    """
    seg_difference_filter: Use elevations and slopes to find bad ATL06 segments
    
    
    Inputs: 
        D6: a granule of ATL06 data, in dictionary format.  Must have entries:
            x_atc, h_li, dh_fit_dx
        
    Returns:
        delta_h_seg: the minimum absolute difference between each segment's endpoints and those of its two neighbors
    """
    h_ep=np.zeros([2, D6['h_li'].size])+np.NaN
    h_ep[0, :]=D6['h_li']-D6['dh_fit_dx']*20
    h_ep[1, :]=D6['h_li']+D6['dh_fit_dx']*20
    delta_h_seg=np.zeros_like(D6['h_li'])
    delta_h_seg[1:]=np.abs(D6['h_li'][1:]-h_ep[1, :-1])
    delta_h_seg[:-1]=np.minimum(delta_h_seg[:-1], np.abs(D6['h_li'][:-1]-h_ep[0, 1:]))
    return delta_h_seg


In [13]:
# find all the files in the directory:
#ATL06_files=glob.glob(os.path.join(data_root, 'PIG_ATL06', '*.h5'))
data_root='/srv/shared/surface_velocity/'
ATL06_files=glob.glob(os.path.join(data_root, 'FIS_ATL06_small', '*.h5'))
D_dict={}
error_count=0
for file in ATL06_files:
    try:
        D_dict[file]=atl06_to_dict(file, '/gt2l', index=slice(0, -1, 25), epsg=3031)
    except KeyError as e:
        print(f'file {file} encountered error {e}')
        error_count += 1
print(f"read {len(D_dict)} data files of which {error_count} gave errors")

read 13 data files of which 0 gave errors


In [15]:
D_2l={}
D_2r={}

# specify the rgt here:
rgt="1092"
# iterate over the repeat cycles
for cycle in ['03','05']:
    for filename in glob.glob(os.path.join(data_root, 'FIS_ATL06_small', f'*ATL06_*_{rgt}{cycle}*_003*.h5')):
        try:
            # read the left-beam data
            D_2l[filename]=atl06_to_dict(filename,'/gt2l', index=None, epsg=3031)
            # read the right-beam data
            D_2r[filename]=atl06_to_dict(filename,'/gt2r', index=None, epsg=3031)
            # plot the locations in the previous plot
            map_ax.plot(D_2r[filename]['x'], D_2r[filename]['y'],'k');  
            map_ax.plot(D_2l[filename]['x'], D_2l[filename]['y'],'k');
        except Exception as e:
            print(f'filename={filename}, exception={e}')

plt.figure();
for filename, Di in D_2l.items():
    #Plot only points that have ATL06_quality_summary==0 (good points)
    hl=plot_segs(Di, ind=Di['atl06_quality_summary']==0, label=f"cycle={Di['cycle']}")
    #hl=plt.plot(Di['x_atc'][Di['atl06_quality_summary']==0], Di['h_li'][Di['atl06_quality_summary']==0], '.', label=f"cycle={Di['cycle']}")
    
plt.legend()
plt.xlabel('x_atc')
plt.ylabel('elevation');

filename=/srv/shared/surface_velocity/FIS_ATL06_small/processed_ATL06_20190608191727_10920311_003_01.h5, exception=name 'map_ax' is not defined
filename=/srv/shared/surface_velocity/FIS_ATL06_small/processed_ATL06_20191207103705_10920511_003_01.h5, exception=name 'map_ax' is not defined


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …